# Scraping the web with BeautifulSoup

We are going to get information out of websites using `requests` and `beautifulsoup`.

## Installation

With conda, you can install the required dependencies with:

```bash
conda install beautifulsoup4 requests
```

or

```bash
python3 -m pip install beautifulsoup4 requests
```


## Basic usage of BeautifulSoup

First, we import the `BeatifulSoup` class:

In [1]:
from bs4 import BeautifulSoup

We load the html source file from disk and pass the source the the BeautifulSoup constructor.

In [2]:
with open("list.html") as f:
    html = f.read()
    document = BeautifulSoup(html, "html.parser")
print(html)

<!doctype html>
<html>
  <body>
    <h2>An Unordered HTML List</h2>

    <ul id="unordered_list" style="color: #069">
      <li>Coffee</li>
      <li>Tea</li>
      <li>Milk</li>
    </ul>

    <h2>An Ordered HTML List</h2>

    <ol id="ordered_list" style="color: #069">
      <li>Coffee</li>
      <li>Tea</li>
      <li>Milk</li>
    </ol>
  </body>
</html>



In [3]:
from IPython.display import HTML

HTML(html)

### Finding tags by name

The document now contains the full html document. We can find the first occuring tag with a specific name with the `find` function. Let's find the first un-ordered list tag:

In [4]:
ulist = document.find("ul")

The result contains all tags contained in the matched tag:

In [5]:
ulist

<ul id="unordered_list" style="color: #069">
<li>Coffee</li>
<li>Tea</li>
<li>Milk</li>
</ul>

The `find_all` function returns **all** tags that match the given tag name. We can use it to get a list of all list items:

In [6]:
items = ulist.find_all("li")
items

[<li>Coffee</li>, <li>Tea</li>, <li>Milk</li>]

Finally, we can loop over all items and extract their contant with the `get_text` function:

In [7]:
for item in items:
    print(repr(item.get_text()))

'Coffee'
'Tea'
'Milk'


Because whitespace is not meaningful in HTML,
it is often useful to strip it when you are getting the content of a tag.
You can do this with `strip=True`

In [8]:
for item in items:
    print(repr(item.get_text(strip=True)))

'Coffee'
'Tea'
'Milk'


Note that `find_all` is **recursive** by default. This means that we could call it the on the full `document` to get the items
of both the ordered and un-ordered lists:

In [9]:
document.find_all("li", recursive=False)

[]

In [10]:
ulist.find_all("li", recursive=False)

[<li>Coffee</li>, <li>Tea</li>, <li>Milk</li>]

A recursive search finds all `li` tags anywhere.

In [11]:
document.find_all("li")

[<li>Coffee</li>,
 <li>Tea</li>,
 <li>Milk</li>,
 <li>Coffee</li>,
 <li>Tea</li>,
 <li>Milk</li>]

### Finding tags by attributes

Sometimes the easiest way to find a tag is by its attribute name. In our examples, both lists have an `id` attribute that uniquely identifies the tables. We can also use the `find*` methods to search for attributes:


In [12]:
document.find(attrs={"id": "unordered_list"})

<ul id="unordered_list" style="color: #069">
<li>Coffee</li>
<li>Tea</li>
<li>Milk</li>
</ul>

### Accessing attributes

The `ul` tag also contains a `style` attribute. Any bs4 tag behaves like a dictionary with attribute names as keys and attribute values as values:

In [13]:
ulist.attrs

{'id': 'unordered_list', 'style': 'color: #069'}

In [14]:
ulist["style"]

'color: #069'

## Downloading a table from Wikipedia

We aim to get a list of countries sorted by their population size:
https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population

First, let's import the required modules:

In [15]:
import re

import dateutil
import requests
from bs4 import BeautifulSoup

This time, we load the html directly from a website using the requests module:

In [16]:
r = requests.get(
    "https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population"
)

The web server returns a status code to indicate if the request was (un-)succesfully.
We use that status-code to check if the page was succesfully loaded:

In [17]:
assert r.status_code == 200

Next, we extract the html source and initiated BeautifulSoup:

In [18]:
html = r.text
document = BeautifulSoup(html, "html.parser")

by looking at the document, we can see that we are interested in first table. So we use `find`:

In [19]:
table = document.find("table", class_="wikitable")

If you are not familiar with html table, read this example first: https://www.w3schools.com/html/tryit.asp?filename=tryhtml_table_intro

In [20]:
print(str(table)[:1024])

<table class="wikitable sortable">
<tbody><tr class="is-sticky">
<th></th>
<th><a href="/wiki/List_of_sovereign_states" title="List of sovereign states">Country</a> / <a href="/wiki/Dependent_territory" title="Dependent territory">Dependency</a></th>
<th>Population</th>
<th>% of the world</th>
<th>Date</th>
<th><span class="nowrap">Source (official or from</span> the <a href="/wiki/United_Nations" title="United Nations">United Nations</a>)</th>
<th class="unsortable">
</th></tr>
<tr>
<td style="text-align:center"><b>–</b>
</td>
<td><b>World</b>
</td>
<td style="text-align:center"><b> 8,038,654,000</b></td>
<td style="text-align:right"><b>100%</b></td>
<td><b><span data-sort-value="000000002023-06-30-0000" style="white-space:nowrap">30 Jun 2023</span></b></td>
<td style="text-align:left"><b>UN projection<sup class="reference" id="cite_ref-unpop_4-0"><a href="#cite_note-unpop-4">[3]</a></sup></b></td>
<td>
</td></tr>
<tr>
<th>1
</th>
<td scope="row" style="text-align:left"><span class="f

At this point, it is a good idea to programatically check that the table contains the correct header:

In [21]:
# assert "Population" in table.find("thead").get_text(strip=True)

In [22]:
rows = table.find_all("tr")

In [23]:
for row in rows[1:]:
    cells = row.find_all(["th", "td"])
    if not cells:
        continue

    cells_text = [cell.get_text(strip=True) for cell in cells]
    (rank, country, population, percentage, updated_at, source, comment) = cells_text

    # Clean population string str("1,404,890,600") -> int(1404890600)
    population = int(population.replace(",", ""))

    # Clean country name
    country = re.findall(r"[\w\s()\.]+", country)[0]

    # Convert percentage to floats
    percentage = float(re.findall(r"[\d\.]+", percentage)[0])

    # Convert updated_at to date object
    updated_at = dateutil.parser.parse(updated_at).date()

    print(f"{rank}, {country}, {population:.2e}, {percentage}%, {updated_at}, {source}")

–, World, 8.04e+09, 100.0%, 2023-06-30, UN projection[3]
1, China, 1.41e+09, 17.6%, 2022-12-31, Official estimate[4]
2, India, 1.39e+09, 17.3%, 2023-03-01, Official projection[5]
3, United States, 3.35e+08, 4.17%, 2023-06-30, National population clock[7]
4, Indonesia, 2.78e+08, 3.46%, 2022-12-31, Official estimate[8]
5, Pakistan, 2.20e+08, 2.74%, 2020-07-01, Official projection[9]
6, Nigeria, 2.19e+08, 2.72%, 2022-07-01, UN projection[3]
7, Brazil, 2.03e+08, 2.53%, 2022-08-01, 2022 census result[10]
8, Bangladesh, 1.70e+08, 2.11%, 2022-06-15, 2022 final census result[11]
9, Russia, 1.46e+08, 1.82%, 2023-01-01, Official estimate[12]
10, Mexico, 1.29e+08, 1.6%, 2022-09-30, National quarterly estimate[13]
11, Japan, 1.24e+08, 1.55%, 2023-05-01, Official estimate[14]
12, Philippines, 1.11e+08, 1.38%, 2023-06-30, National population clock[15]
13, Ethiopia, 1.05e+08, 1.31%, 2022-07-01, National annual projection[16]
14, Egypt, 1.02e+08, 1.27%, 2021-07-01, Official estimate[17]
15, Vietnam, 9

ValueError: not enough values to unpack (expected 7, got 6)

**Attention**: Beautiful Soup does not execute Javascript. This means that you the code in your browser inspector might look a bit different from the original HTML source code. 

# Another example of downloading a Wikipedia table 

Let's consider another table in a Wikipedia page. This page has a lot more tables, so one challenge will be to pick the right table

https://en.wikipedia.org/wiki/Serena_Williams


We are interested in extracting these two tables:

![Target Wikipedia tables](pictures/wiki_tables.png)

**Exercise**: 

Find the tables on a page by locating heading and using `.find_next()`


We begin by downloading the webpage and instatiating the BeautifulSoup object:

In [ ]:
r = requests.get("https://en.wikipedia.org/wiki/Serena_Williams")
document = BeautifulSoup(r.text, "html.parser")

This page contains a lot of tables without specific attributes that would make it easy to find our table of interest. Further, the same headings of the tables are used for multiple tables, making it difficult to find a table just by its headings:

In [ ]:
len(document.find_all("table"))

Therefore, we choose another strategy.

First, we find the tag with class `mw-headling` whose `string` content _starts with_ `Singles`.
Then we find the _next_ table using `heading_element.find_next(...)`:

In [ ]:
document.find_all(class_="mw-headline", string=re.compile("^Singles"))

In [ ]:
# string class
singles_heading = document.find(class_="mw-headline", string=re.compile("^Singles"))
singles_heading

In [ ]:
singles_heading.find_next("table")

Now, our tables of interest are the first two result tables for "Singles" and "Women's doubles". We write a small helper function that returns a table with a given heading:

In [ ]:
def find_table_with_heading(document, heading_pat):
    heading_element = document.find(class_="mw-headline", string=heading_pat)
    table = heading_element.find_next("table")
    return table

In [ ]:
singles_table = find_table_with_heading(document, re.compile("^Singles"))
# print headers
headings = singles_table.find_all("th")
[th.get_text(strip=True) for th in headings]

Next, we can find the table after the heading "Women's doubles"

In [ ]:
doubles_table = find_table_with_heading(document, re.compile(r"^Women's doubles"))
# print headers
headings = doubles_table.find_all("th")
[th.get_text(strip=True) for th in headings]

## Exercise:

- Iterate through the rows
- convert year to integer (or date)
- strip note '(12)' from event, so the same event has the same string
- load into pandas DataFrame (more on pandas in a later lecture)

In [ ]:
re.sub?

In [ ]:
data = []
for row in singles_table.find_all("tr"):
    cells = row.find_all("td")
    if not cells:
        # e.g. header row
        continue
    values = [td.get_text(strip=True) for td in cells]
    # convert year to int
    values[1] = int(values[1])
    # strip trailing (n) notes from events
    values[2] = re.sub(r"\s*\(.+\)$", "", values[2])
    data.append(values)
data

When data is in this form, we can convert it into a DataFrame with pandas.

You'll learn more about pandas next week.

In [ ]:
import pandas as pd

headings = [th.get_text(strip=True) for th in singles_table.find_all("th")]
df = pd.DataFrame(data, columns=headings)
df

With pandas, we can filter this data, group it, and plot interesting relationships.

Pandas `groupby` is an interesting operation for performing aggregations,
e.g. counting the wins/losses by year and result:

In [ ]:
df.Result.value_counts()

In [ ]:
results_by_year = df.groupby(["Year", "Result"]).Tournament.count().unstack()
results_by_year

Which we can now plot

In [ ]:
results_by_year.plot(kind="bar")

Is there any significance to the court?

In [ ]:
results_by_surface = df.groupby(["Surface", "Result"]).Tournament.count().unstack()
results_by_surface

In [ ]:
results_by_surface.plot(kind="bar")

We can even filter to e.g. select opponents who Williams faced at least twice

In [ ]:
results_by_op = df.groupby(["Opponents", "Result"]).Tournament.count().unstack()
results_by_op

In [ ]:
# we can exclude opponents only met once:
results_by_op = results_by_op.fillna(0)
results_by_op

In [ ]:
(results_by_op.Win + results_by_op.Loss) > 1

In [ ]:
multiple_meetings = results_by_op[(results_by_op.Win + results_by_op.Loss) > 1]
multiple_meetings.plot(kind="bar")

# Exercise:

Find images on the UiO page

1) Go to https://en.wikipedia.org/wiki/University_of_Oslo 
2) Download the content from the site using BeautifulSoup and requests
3) Search for all images (using `images = document.find_all('img')`) and print out the content
4) Include only images with the attribute `class_="thumbimage"` in your list of images.
5) Print out a list of the value of the "src" attribute for the images in 4. 
6) See if you can display an image by pasting a result from 5 into your web-browser.

In [ ]:
r = requests.get("https://en.wikipedia.org/wiki/University_of_Oslo")
html = r.text
print(html[:400])

In [ ]:
document = BeautifulSoup(html, "html.parser")

In [ ]:
images = document.find_all("img", class_="thumbimage")
len(images)

In [ ]:
for image in images:
    print(f"https:{image['src']}")

In [ ]:
from IPython.display import HTML, display

In [ ]:
for image in images:
    src_url = f"https:{image['src']}"
    html = HTML(f'<img src="{src_url}">')
    display(html)